# Микродиахроническое исследование русских приставок методами дистрибутивной семантики
## Автор: Елизавета Клыкова, БКЛ181
### Извлечение всех вхождений нужных глаголов
1. Выбрать из базы предложения нужных периодов
2. Для каждого исследуемого глагола получить набор токенов с грамматическими характеристиками
3. Сохранить в pickle-файлы

#### Импорт модулей

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import pickle
import pymongo
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter

**Подключение к базе**

In [3]:
client = pymongo.MongoClient('localhost', 27017)
db = client['thesis']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'thesis')

In [4]:
sentences = db.sentences
lemmas = db.lemmas
tokens = db.tokens

#### Получение списка глаголов

In [5]:
pref_df = pd.read_csv('prefixes_and_lemmas_dropna.tsv', sep='\t')
pref_df

,prefix,lemma,abs_freq,abs_freq0,abs_freq1,abs_freq2,freq,freq0,freq1,freq2
0,бе[зс],бездействовать,541,155,255,131,2.194,2.147,2.741,1.609
1,бе[зс],бездельничать,308,41,169,98,1.249,0.568,1.817,1.204
2,бе[зс],безмолвствовать,726,361,191,174,2.944,5.000,2.053,2.138
3,бе[зс],безобразить,149,99,43,7,0.604,1.371,0.462,0.086
4,бе[зс],безобразничать,306,110,136,60,1.241,1.524,1.462,0.737
...,...,...,...,...,...,...,...,...,...,...
8429,у,ущемляться,62,8,16,38,0.251,0.111,0.172,0.467
8430,у,ущипывать,510,147,222,141,2.068,2.036,2.386,1.732
8431,у,уязвлять,1305,606,411,288,5.291,8.393,4.418,3.538
8432,у,уяснять,1803,677,740,386,7.311,9.377,7.954,4.742


In [6]:
# list of dicts
pref_dict = pref_df.to_dict(orient='records')

In [7]:
verbs_to_search = sorted(set([v['lemma'] for v in pref_dict]))

#### Поиск глагольных употреблений в базе

In [8]:
# досоветский период

presov_verb_dict = {verb: [] for verb in verbs_to_search}

for sentence in tqdm(sentences.find({'period': 0},
                                    {'token_info': True, '_id': False})):
    verb_list = [token for token in sentence['token_info']
                 if token['pos'] == 'V']
    for verb in verb_list:
        if verb['lemma'] in verbs_to_search:
            presov_verb_dict[verb['lemma']].append({'token': verb['token'],
                                                    'gram': verb['gram'].split('|')[0]})

0it [00:00, ?it/s]

12:80: E501 line too long (88 > 79 characters)


In [9]:
with open('presov_verb_dict.pickle', 'wb') as f:
    pickle.dump(presov_verb_dict, f)

In [10]:
# советский период

sov_verb_dict = {verb: [] for verb in verbs_to_search}

for sentence in tqdm(sentences.find({'period': 1},
                                    {'token_info': True, '_id': False})):
    verb_list = [token for token in sentence['token_info']
                 if token['pos'] == 'V']
    for verb in verb_list:
        if verb['lemma'] in verbs_to_search:
            sov_verb_dict[verb['lemma']].append({'token': verb['token'],
                                                 'gram': verb['gram'].split('|')[0]})

0it [00:00, ?it/s]

12:80: E501 line too long (85 > 79 characters)


In [11]:
with open('sov_verb_dict.pickle', 'wb') as f:
    pickle.dump(sov_verb_dict, f)

In [12]:
# постсоветский период

postsov_verb_dict = {verb: [] for verb in verbs_to_search}

for sentence in tqdm(sentences.find({'period': 2},
                                    {'token_info': True, '_id': False})):
    verb_list = [token for token in sentence['token_info']
                 if token['pos'] == 'V']
    for verb in verb_list:
        if verb['lemma'] in verbs_to_search:
            postsov_verb_dict[verb['lemma']].append({'token': verb['token'],
                                                     'gram': verb['gram'].split('|')[0]})

0it [00:00, ?it/s]

12:80: E501 line too long (89 > 79 characters)


In [13]:
with open('postsov_verb_dict.pickle', 'wb') as f:
    pickle.dump(postsov_verb_dict, f)